# Import data from json

In [1]:
import json
with open("dataset/tradesy.json", "r") as f:
    user_transactions = json.loads(f.read())

# Stat:
- See how many products
- Which fields, and which should be concerned

In [2]:
user_transactions[:3]

[{'uid': '1',
  'lists': {'sold': ['3', '2'], 'selling': [], 'want': [], 'bought': []}},
 {'uid': '2',
  'lists': {'sold': ['104', '103', '102'],
   'selling': [],
   'want': [],
   'bought': ['466', '459', '457', '449']}},
 {'uid': '3',
  'lists': {'sold': ['845', '833', '829'],
   'selling': [],
   'want': [],
   'bought': ['874',
    '861',
    '860',
    '857',
    '852',
    '850',
    '847',
    '842',
    '160',
    '143',
    '142',
    '141',
    '140',
    '139',
    '93',
    '88',
    '87',
    '84',
    '81',
    '70',
    '66',
    '63',
    '61',
    '60']}}]

In [4]:
print("Number of users:", len(user_transactions))

Number of users: 128152


In [5]:
print("Fields of each user records:", ", ".join(user_transactions[0].keys()))
print("Fields of each tx lists:", ", ".join(user_transactions[0]["lists"].keys()))

Fields of each user records: uid, lists
Fields of each tx lists: sold, selling, want, bought


In [59]:
import numpy as np

def get_unique_items(user_transactions):
    item_ids = []
    for tx in user_transactions:
        if not len(tx["lists"]["want"]):
            continue
        item_ids.extend(tx["lists"]["want"])
        item_ids.extend(tx["lists"]["selling"])
    unique_ids = np.unique(item_ids, return_counts=True)
    return unique_ids

unique_ids, counts = get_unique_items(user_transactions)
print("Unique items:", len(unique_ids))
print("=> Size to store: ~", len(unique_ids) * len(user_transactions) / 1024 / 1024, ", MB > 8 GB RAM")

Unique items: 227856
=> Size to store: ~ 27847.482788085938 , MB > 8 GB RAM


# Recommender system focus on "sold"


In [77]:
# Transform data: transaction jsons => table
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder 

userToSold = {}
focus_transactions = user_transactions[:300]
sold_list = []
for tx in focus_transactions:
    if len(tx["lists"]["sold"]) == 0:
        continue 
    userToSold[tx["uid"]] = list(map(int, tx["lists"]["sold"]))
    userToSold[tx["uid"]] = list(map(int, tx["lists"]["bought"]))
    sold_list.append(userToSold[tx["uid"]])

encoder = TransactionEncoder()
sold_list_matrix = encoder.fit_transform(sold_list).astype("int")

df = pd.DataFrame(sold_list_matrix, columns=encoder.columns_)

In [79]:
from mlxtend.frequent_patterns import apriori
fq_itemset = apriori(df, min_support=0.7)

print(fq_itemset)
print("Number of rules:", len(fq_itemset))

Empty DataFrame
Columns: [support, itemsets]
Index: []
Number of rules: 0
